# 神経データに関する隠れ空間モデル

多くの科学の分野は社会ネットワーク、統計物理におけるネットワーク、生物学的ネットワーク、情報ネットワークなど
ネットワークデータの研究を含みます(Goldenberg, Zheng, Fienberg, & Airoldi, 2010; Newman, 2010)。

ネットワークのノード(接点)に対してその結合のパターンから何を知ることができるでしょうか。
隠れ空間モデルを使ってその研究を始めることができます(Hoff, Raftery, & Handcock, 2002)。

隠れ空間モデルはネットワークのノードを隠れた空間に埋め込みます。
2つのノードの間にあるエッジ(辺)ができる尤度は隠れた空間内での距離によって決まります。

ここで我々は神経科学のネットワークデータを解析します。Jupyter notebookでの原文は [ここ](http://nbviewer.jupyter.org/github/blei-lab/edward/blob/master/notebooks/latent_space_models.ipynb)にあります。

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal, Poisson
from observations import celegans

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/apple/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# データ

使用するデータは[Mark Newmanのリポジトリ](http://www-personal.umich.edu/~mejn/netdata/)のものです。
これは線虫[C. Elegans](https://en.wikipedia.org/wiki/Caenorhabditis_elegans)の神経ネットワークを表現する
重みづけられた有向のネットワークでWhite, Southgate, Thomson, & Brenner (1986)の実験データ
Watts & Strogatz (1998)がまとめたものです。

この神経ネットワークは約300個のニューロンで構成されています。それぞれのニューロン間の結合は
その強さを表した重み(正の整数)に対応します。

まずデータをロードします。

In [63]:
import os
from observations.util import maybe_download_and_extract

"""Load the neural network of the worm C. Elegans [@watts1998collective].                                                                                                                                                                                                  
  The neural network consists of around 300 neurons. Each connection                                                                                                                                                                                                         
  between neurons is associated with a weight (positive integer)                                                                                                                                                                                                             
  capturing the strength of the connection.                                                                                                                                                                                                                                  
                                                                                                                                                                                                                                                                             
  Args:                                                                                                                                                                                                                                                                      
    path: str.                                                                                                                                                                                                                                                               
      Path to directory which either stores file or otherwise file will                                                                                                                                                                                                      
      be downloaded and extracted there. Filename is `celegansneural.gml`.                                                                                                                                                                                                   
                                                                                                                                                                                                                                                                             
  Returns:                                                                                                                                                                                                                                                                   
    Adjacency matrix as a np.darray `x_train` with 297 rows and 297                                                                                                                                                                                                          
    columns.                                                                                                                                                                                                                                                                 
"""
def celegans(path):
    import networkx as nx
    path = os.path.expanduser(path)
    filename = 'celegansneural.gml'
    if not os.path.exists(os.path.join(path, filename)):
        url = 'http://www-personal.umich.edu/~mejn/netdata/celegansneural.zip'
        maybe_download_and_extract(path, url)

    graph = nx.read_gml(os.path.join(path, filename))
    x_train = np.zeros([graph.number_of_nodes(), graph.number_of_nodes()],
                     dtype=np.int)
    for i, j,k in graph.edges:
        try:
            x_train[int(i),int(j)] = int(graph[i][j][0]['value']) #左辺のintが必要
        except:
            pass
    return x_train #,graph

In [64]:
#from observations import celegans
x_train = celegans("data")

# モデル

ニューロンに関してその結合パターンから何を知ることができるでしょうか。
隠れ空間モデル(Hoff et al., 2002)を使うことで我々は個々のニューロン間の類似性を捉えられる隠れ空間への埋め込みを学習することができます。

それぞれのニューロンnはネットワークのノードで隠れ空間内での位置$z_n\in\mathbb{R}^K$が対応します。それぞれの隠れ空間における位置にガウシアン事前分布を設定します。

ノードi,j間のエッジのオッズの対数はノードの隠れた表現のユークリッド距離$|z_i- z_j|$に比例します。ここでエッジの重み ($Y_{ij}$) をポアソン尤度でモデル化します。その割合は隠れた空間での距離の逆数になります。生成モデルは以下のようになります。

1. 
$n=1,\ldots,N$の各ノードに対し,
\begin{align}
z_n \sim N(0,I).
\end{align}
2. 
各エッジ $(i,j)\in\{1,\ldots,N\}\times\{1,\ldots,N\}$　に対し,
\begin{align}
Y_{ij} \sim \text{Poisson}\Bigg(\frac{1}{|z_i - z_j|}\Bigg).
\end{align}

Edwardでは以下のようにモデル化されます。

In [15]:
N = x_train.shape[0]  #データ点の数
K = 3  # 隠れた次元

z = Normal(loc=tf.zeros([N, K]), scale=tf.ones([N, K]))

# N x N 距離行列を計算する
# 1. ベクトル [||z_1||^2, ||z_2||^2, ..., ||z_N||^2]　を作りそれを並べてN個の行を作る。
xp = tf.tile(tf.reduce_sum(tf.pow(z, 2), 1, keep_dims=True), [1, N])
# 2. 　(i, j) 成分が ||z_i||^2 + ||z_j||^2- 2 z_i^T z_j　のN x N 行列　
xp = xp + tf.transpose(xp) - 2 * tf.matmul(z, z, transpose_b=True)
# 3. ペアワイズ距離の逆数を取り対角成分に沿って0に近くする
xp = 1.0 / tf.sqrt(xp + tf.diag(tf.zeros(N) + 1e3))

x = Poisson(rate=xp)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


# 推測

最大事後分布推定はEdwardでは単純です。2つの事柄、インスタンス化とそれを走らせる処理が要求されます。

In [66]:
inference = ed.MAP([z], data={x: x_train})

/usr/local/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/usr/local/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


[MAP 推定のチュートリアル](http://edwardlib.org/tutorials/map)を参照

代わりに変分推論を走らせることもできます。これには変分モデルと`KLqp`をインスタンス化することが必要になります。

In [ ]:
# Alternatively, run
# qz = Normal(loc=tf.Variable(tf.random_normal([N * K])),
#             scale=tf.nn.softplus(tf.Variable(tf.random_normal([N * K]))))
# inference = ed.KLqp({z: qz}, data={x: x_train})

[変分推論](http://edwardlib.org/tutorials/variational-inference)のチュートリアルも参照

最後に以下の行で推論処理を2500イテレーションだけ走らせます。

In [67]:
inference.run(n_iter=2500)

2500/2500 [100%] ██████████████████████████████ Elapsed: 11s | Loss: 30781.020


In [69]:
inference.run(n_iter=10000)

10000/10000 [100%] ██████████████████████████████ Elapsed: 54s | Loss: 30856.160


## Acknowledgments

この原文の最初のバージョンを書いたMaja Rudolphに感謝します。